In [7]:
# Importing necessary libraries
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the number of qubits
n_qubits = 3

# Initialize a device
dev = qml.device('default.qubit', wires=n_qubits, shots=10)

# Define the initial state preparation function
def initial_state(alpha, beta):
    qml.RY(2 * np.arccos(alpha), wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])

# Define the weak error function
def apply_weak_error():
    # Randomly select one of the three qubits
    qubit_to_flip = random.randint(0, 2)
    prob = 0.1
    if random.random() < prob:
        qml.PauliX(wires=qubit_to_flip)

def plot_circuit_and_probs(circuit, *args, description=""):
    # Execute the circuit to obtain the probabilities
    probs = circuit(*args)
    # Calculate the squared magnitude of each probability amplitude
    squared_probs = np.abs(probs)**2
    # Draw the circuit
    fig, ax = qml.draw_mpl(circuit, expansion_strategy="device")(*args)
    plt.show()
    # Print the probabilities
    print(f"Probabilities of states after {description}:")
    for i, p in enumerate(squared_probs):
        print(f"State |{i:03b}⟩: {p:.4f}")
    print("\n")

def apply_Z1_Z2():
    qml.PauliZ(wires=0)
    qml.PauliZ(wires=1)

def apply_Z1_Z3():
    qml.PauliZ(wires=0)
    qml.PauliZ(wires=2)

# Define the quantum node
@qml.qnode(dev)
def circuit1():
    initial_state()
    return qml.state()

# Define the quantum node
@qml.qnode(dev)
def circuit2():
    initial_state()
    apply_weak_error()
    return qml.state()

# Define the quantum node for measuring the expectation value of Z1Z2 after the weak error
@qml.qnode(dev)
def circuit_Z1Z2():
    initial_state()
    apply_weak_error()
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

# Define the quantum node for measuring the expectation value of Z1Z3 after the weak error
@qml.qnode(dev)
def circuit_Z1Z3():
    initial_state()
    apply_weak_error()
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(2))

@qml.qnode(dev)
def circuit2(alpha, beta):
    initial_state(alpha, beta)
    apply_weak_error()
    
    # Measure the stabilizers
    Z1Z2 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(1))
    Z1Z3 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(2))
    
    # Define error correction based on the stabilizer measurements
    if Z1Z2 == 1 and Z1Z3 == 1:
        pass
    elif Z1Z2 == -1 and Z1Z3 == -1:
        qml.PauliX(wires=0)
    elif Z1Z2 == -1 and Z1Z3 == 1:
        qml.PauliX(wires=1)
    elif Z1Z3 == 1 and Z1Z2 == -1:
        qml.PauliX(wires=2)
    
    # Measure the corrected state after error correction
    corrected_state = qml.probs(wires=[0, 1, 2])
    
    # Return stabilizer measurements and corrected state
    return Z1Z2, Z1Z3, corrected_state

# Parameters for the state |ψ⟩ = α|000⟩ + β|111⟩
alpha = 0.4
beta = np.sqrt(1 - alpha ** 2)




Z1Z2, Z1Z3, corrected_state = circuit2(alpha, beta)

plot_circuit_and_probs(circuit2, alpha, beta, description="weak error correction")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.